In [1]:
# -*- coding: utf-8 -*-
import ee
import eefolium as basicgeemap
#import geemap
print("start inializing")
ee.Initialize(project='ee-channingtong')
print("initialized")
# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/061/MOD11A1')
# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')
# Import the YZB boundary.
boundary = ee.FeatureCollection('projects/ee-channingtong/assets/YZBboundary')
geometry = boundary.first().geometry()
print("imported")

inital_date = '2022-01-01'
final_date = '2023-01-01'
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(inital_date, final_date).filterBounds(geometry)
m_scale = 1000  # scale in meters
elv = elv.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=m_scale)
print('Ground elevation average:', elv.get('elevation').getInfo(), 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_info = lst.getInfo();
print("lst count", len(lst_info['features']))
print("data range", lst_info['features'][0]['properties']['system:time_start'], lst_info['features'][-1]['properties']['system:time_start'])
#lst = lst.map(lambda img: img.clip(boundary))
#lst_info = lst.mean().get('LST_Day_1km').getInfo()
#print(lst.getInfo())
#print('Average daytime LST average:', round(lst.mean().get('LST_Day_1km').getInfo()*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
# lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
# print('Land cover value average is:', lc_urban_point)
print("reduced")

temperature = lst.first().select('LST_Day_1km')
celsius = temperature.multiply(0.02).subtract(273.15)
Map = basicgeemap.Map()
Map.addLayer(celsius,
             {'min': -20, 'max': 50, 
              'palettes': ['blue', 'green', 'yellow', 'orange', 'red']
              },'temperature')
'''
Map.addLayer(boundary,
          {
        'color': 'blue',  # Outline color
        'fillColor': 'lightblue',  # Fill color (if applicable)
        'width': 2  # Outline width
          },
             'boundary')
'''
Map.centerObject(boundary, 8)
Map

start inializing
initialized
imported
Ground elevation average: 143.17068051689284 m
lst count 353
data range 1640995200000 1672444800000
reduced


In [2]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df